In [148]:
!kaggle competitions download -c DATAHACK-Yalidine-Challenge

... resuming from 8998779 bytes (1683223 bytes left) ...
100%|███████████████████████████████████████| 10.2M/10.2M [00:02<00:00, 624kB/s]
100%|███████████████████████████████████████| 10.2M/10.2M [00:02<00:00, 606kB/s]


In [4]:
!unzip DATAHACK-Yalidine-Challenge.zip

Archive:  DATAHACK-Yalidine-Challenge.zip
  inflating: test.csv                
  inflating: train_data.csv          


In [2]:
from sklearn.preprocessing import LabelEncoder

In [1]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test.csv')

In [3]:
!nvidia-smi

Fri Mar  8 08:56:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P0              14W /  80W |      6MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [224]:
df_train.head()

,Tracking,date expédition,date première tentative,date livraison/échec,date dernier statut,ID expediteur,depart commune,depart wilaya,stop desk?,destination commune,destination wilaya,dernier statut
0,yal-ABC240400,2024-01-10 18:18:53,2024-01-12 16:46:52,2024-01-12 16:46:52,2024-01-12 16:46:52,expyal-11571,Oued Smar,Alger,oui,El Menia,Ghardaïa,Livré
1,yal-ABC089640,2024-01-04 18:55:07,2024-01-05 14:50:19,2024-01-05 14:50:19,2024-01-05 14:50:19,expyal-95,Oued Smar,Alger,non,Médéa,Médéa,Livré
2,yal-ABC004222,2024-01-02 16:47:55,2024-01-04 08:39:02,2024-01-05 09:47:54,2024-01-05 09:47:54,expyal-571,Taher,Jijel,oui,Tizi Ouzou,Tizi Ouzou,Livré
3,yal-ABC003797,2024-01-01 17:27:33,2024-01-02 10:10:15,2024-01-02 10:14:20,2024-01-02 10:14:20,expyal-13280,Cheraga,Alger,non,Tlemcen,Tlemcen,Livré
4,yal-ABC132873,2024-01-07 14:45:47,2024-01-08 10:49:38,2024-01-08 10:49:38,2024-01-08 10:49:38,expyal-1623,Khenchela,Khenchela,non,Bordj Bou Arreridj,Bordj Bou Arreridj,Livré


In [7]:
df_train.describe()

,Tracking,date expédition,date première tentative,date livraison/échec,date dernier statut,ID expediteur,depart commune,depart wilaya,stop desk?,destination commune,destination wilaya,dernier statut
count,271481,271481,270360,270332,271481,271481,271481,271481,271481,271481,271481,271481
unique,271481,42188,153353,194983,184807,15512,116,48,2,1365,48,12
top,yal-ABC240400,2024-01-08 15:10:17,2024-01-12 00:03:28,2024-01-15 11:54:57,2024-01-20 17:34:41,expyal-17226,Oued Smar,Alger,oui,Constantine,Alger,Livré
freq,1,354,30,18,703,17384,58347,115959,144903,9204,31955,236293


In [8]:
df_train.isna().sum()

Tracking                      0
date expédition               0
date première tentative    1121
date livraison/échec       1149
date dernier statut           0
ID expediteur                 0
depart commune                0
depart wilaya                 0
stop desk?                    0
destination commune           0
destination wilaya            0
dernier statut                0
dtype: int64

In [13]:
df_train['dernier statut'].value_counts()

dernier statut
3     236293
8      33854
10       542
6        295
11       248
4         63
1         56
0         51
2         30
9         29
5         12
7          8
Name: count, dtype: int64

In [12]:
df_train['dernier statut'][0]

'Livré'

In [15]:
df_train.columns[5:]

Index(['ID expediteur', 'depart commune', 'depart wilaya', 'stop desk?',
       'destination commune', 'destination wilaya', 'dernier statut'],
      dtype='object')

In [3]:
def process(data):
    columns = data.columns
    date_columns = ['date expédition' ,'date première tentative' ,'date livraison/échec' ,'date dernier statut']
    LE = LabelEncoder()
    gg = []
    for le_col in columns[5:]:
      data[le_col] = LE.fit_transform(data[le_col])
    for clm in date_columns:
        data[clm] = pd.to_datetime(data[clm])
        data['year'+str(clm)] = data[clm].dt.year
        data['year'+str(clm)] = data['year'+str(clm)].astype(float)
        data['month'+str(clm)] = data[clm].dt.month
        data['day'+str(clm)] = data[clm].dt.day
        data['weekday'+str(clm)] = data[clm].dt.weekday
        data['weekofyear'+str(clm)] = data[clm].dt.isocalendar().week
        data['hour'+str(clm)] = data[clm].dt.hour
        data['day_hour'+str(clm)]  = data['day'+str(clm)].astype(str) + '-' + data['hour'+str(clm)].astype(str)
        data['month_day'+str(clm)] = data['month'+str(clm)].astype(str) + '-' + data['day'+str(clm)].astype(str)
        LE.fit_transform(data['day_hour'+str(clm)])
        LE.fit_transform(data['month_day'+str(clm)])
    LE = LabelEncoder()
    data['day_hourdate expédition'] = LE.fit_transform(data['day_hourdate expédition'])
    data['month_daydate expédition'] = LE.fit_transform(data['month_daydate expédition'])
    data['day_hourdate première tentative'] = LE.fit_transform(data['day_hourdate première tentative'])
    data['month_daydate première tentative'] = LE.fit_transform(data['month_daydate première tentative'])
    data['day_hourdate livraison/échec'] = LE.fit_transform(data['day_hourdate livraison/échec'])
    data['month_daydate livraison/échec'] = LE.fit_transform(data['month_daydate livraison/échec'])
    data['day_hourdate dernier statut'] = LE.fit_transform(data['day_hourdate dernier statut'])
    data['month_daydate dernier statut'] = LE.fit_transform(data['month_daydate dernier statut'])
    return data
trying = process(df_train)

In [6]:
train = trying.dropna()


In [7]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

In [8]:
X = train.drop(['Tracking','dernier statut','date livraison/échec','date dernier statut', 'date expédition', 'date première tentative'], axis=1)
y = train['dernier statut']
train_x, test_x ,train_y, test_y = train_test_split(X,y, random_state=28, test_size=0.3)

#### LGBM

In [10]:
from lightgbm import LGBMClassifier

model_lgbm = LGBMClassifier(device = 'gpu',objective='multiclass', n_estimators=2000, num_leaves=43, learning_rate=0.001,seed= 48)

In [11]:
model_lgbm.fit(train_x, train_y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1872
[LightGBM] [Info] Number of data points in the train set: 189232, number of used features: 30
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 30 dense feature groups (5.77 MB) transferred to GPU in 0.006551 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -9.154997
[LightGBM] [Info] Start training from score -0.134148
[LightGBM] [Info] Start training from score -8.322088
[LightGBM] [Info] Start training from score -10.204819
[LightGBM] [Info] Start training from score -6.837523
[LightGBM] [Info] Start training from score -10.358970
[LightGBM] [Info] Start training from score -2.087080


KeyboardInterrupt: 

In [57]:
y_prob = model.predict(test_x)

In [58]:
accuracy = accuracy_score(test_y, y_prob)
print('Accuracy:', accuracy)

Accuracy: 0.9975566333108232


#### Ensemble Learning

24

In [21]:
from sklearn.utils.class_weight import compute_class_weight
label_encoded_targets = train['dernier statut']
label_encoded_targets


unique_classes = np.unique(label_encoded_targets)
unique_classes


class_weights = compute_class_weight(class_weight="balanced", classes=unique_classes, y=label_encoded_targets)
class_weights



class_weights_param = {key: value for key, value in zip(unique_classes, class_weights)}
class_weights_param



{0: 33791.5,
 1: 1299.673076923077,
 3: 0.14300798591567176,
 4: 572.7372881355932,
 5: 3379.15,
 6: 114.93707482993197,
 7: 4223.9375,
 8: 1.0044139939957792}

In [24]:
# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, plot_importance
from catboost import CatBoostClassifier
lgbm_params: dict = {
    "class_weight": class_weights_param, # Balanced class weight
    "objective": "multiclass",          # Objective function for the model
    "metric": "multi_logloss",          # Evaluation metric
    "verbosity": -1,                    # Verbosity level (-1 for silent)
    "boosting_type": "gbdt",            # Gradient boosting type
    "random_state": 42,       # Random state for reproducibility
    "num_class": 12,                     # Number of classes in the dataset
    'n_estimators': 752,
    'learning_rate': 0.005613916463106189,
    'max_depth': 6,
    'num_leaves': 252,
    'subsample': 0.6045538705062335,
    'colsample_bytree': 0.866501494211133,
    'colsample_bynode': 0.5443098861233086,
    'reg_alpha': 0.0024787490924597882,
    'reg_lambda': 3.5334079815178954,
    'min_split_gain': 0.05539710161546875,
}


xgb_params: dict = {
    'class_weight': class_weights_param,
    'objective':'multi:softmax',
    'n_estimators': 829,
    'learning_rate': 0.010260565670497695,
    'gamma': 0.16282691057583543,
    'reg_alpha': 0.010492176264956674,
    'reg_lambda': 0.437536781187624,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6971737476610285,
    'colsample_bytree': 0.5115061295805807,
    'random_state': 345,
}

cat_params: dict = {
    'class_weights': class_weights_param,
    'learning_rate': 0.13762007048684638,
    'depth': 5, 
    'l2_leaf_reg': 5.285199432056192,
    'bagging_temperature': 0.6029582154263095,
    'random_seed': 42,
    'verbose': False,
    'iterations':1000,
}
estimators = [
    ('XGB', XGBClassifier(**xgb_params)),
    ('LGBM', LGBMClassifier(**lgbm_params)),
    ('CAT', CatBoostClassifier(**cat_params))
]

In [35]:
import optuna
TUNE = True
# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the LGBMClassifier
    param = {
        "class_weight": class_weights_param,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 42,
        "num_class": 7,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03),
        "n_estimators": trial.suggest_int("n_estimators", 400, 600),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.005, 0.025),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.02, 0.06),
        "max_depth": trial.suggest_int("max_depth", 6, 14),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
    }

    # Create an instance of LGBMClassifier with the suggested parameters
    lgbm_classifier = LGBMClassifier(**param)
    
    # Fit the classifier on the training data
    lgbm_classifier.fit(X_train, y_train)

    # Evaluate the classifier on the test data
    score = lgbm_classifier.score(X_test, y_test)
    
    print(f'SCORE: {score}')

    return score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

# If TUNE
if TUNE:
    # Run the optimization process
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=150)

    # Get the best parameters after optimization
    best_params = study.best_params

[I 2024-03-08 13:53:16,349] A new study created in memory with name: no-name-510c0c72-b458-4933-87d4-12788e4416ca
[I 2024-03-08 13:53:41,096] Trial 0 finished with value: 0.9981689385392198 and parameters: {'learning_rate': 0.01749080237694725, 'n_estimators': 591, 'lambda_l1': 0.019639878836228104, 'lambda_l2': 0.04394633936788146, 'max_depth': 7, 'colsample_bytree': 0.3935967122017216, 'subsample': 0.5290418060840998, 'min_child_samples': 45}. Best is trial 0 with value: 0.9981689385392198.


SCORE: 0.9981689385392198


[I 2024-03-08 13:54:07,467] Trial 1 finished with value: 0.9983908853829507 and parameters: {'learning_rate': 0.022022300234864175, 'n_estimators': 542, 'lambda_l1': 0.005411689885916049, 'lambda_l2': 0.05879639408647977, 'max_depth': 13, 'colsample_bytree': 0.4274034664069657, 'subsample': 0.5909124836035503, 'min_child_samples': 17}. Best is trial 1 with value: 0.9983908853829507.


SCORE: 0.9983908853829507


[I 2024-03-08 13:54:26,586] Trial 2 finished with value: 0.9981134518282871 and parameters: {'learning_rate': 0.016084844859190754, 'n_estimators': 505, 'lambda_l1': 0.013638900372842315, 'lambda_l2': 0.03164916560792168, 'max_depth': 11, 'colsample_bytree': 0.3836963163912251, 'subsample': 0.6460723242676091, 'min_child_samples': 25}. Best is trial 1 with value: 0.9983908853829507.


...

SCORE: 0.9984833632345054


[I 2024-03-08 15:01:20,498] Trial 148 finished with value: 0.9984648676641944 and parameters: {'learning_rate': 0.027438912744175807, 'n_estimators': 510, 'lambda_l1': 0.011064501376112977, 'lambda_l2': 0.023417659692110168, 'max_depth': 11, 'colsample_bytree': 0.44011370839249614, 'subsample': 0.7447291975401845, 'min_child_samples': 27}. Best is trial 11 with value: 0.9985203543751272.


SCORE: 0.9984648676641944


[I 2024-03-08 15:01:41,046] Trial 149 finished with value: 0.9984833632345054 and parameters: {'learning_rate': 0.02939952015683276, 'n_estimators': 562, 'lambda_l1': 0.006403309720918303, 'lambda_l2': 0.021991295614347622, 'max_depth': 14, 'colsample_bytree': 0.36727293233264063, 'subsample': 0.5236900247346906, 'min_child_samples': 32}. Best is trial 11 with value: 0.9985203543751272.


SCORE: 0.9984833632345054


In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier
WEIGHT_TUNE = True
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the weighted ensemble
    
    obj_estimators = [
        ('XGB', XGBClassifier(**xgb_params)),
        ('LGBM', LGBMClassifier(**lgbm_params)),
        ('CAT', CatBoostClassifier(**cat_params))
    ]
    
    voting_classifier = VotingClassifier(
        estimators=obj_estimators,
        voting='soft',
        weights=[
            trial.suggest_float('XGB_Weight', 1.00, 9.00),
            trial.suggest_float('LGBM_Weight', 0.25, 5.00),
            trial.suggest_float('CAT_Weight', 0.25, 2.00),
        ],
    )
    
    voting_classifier.fit(X_train, y_train)
    
    predict_probs = voting_classifier.predict_proba(X_test)
    
    auc_score = roc_auc_score(y_test, predict_probs, multi_class='ovr')

    return auc_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
weight_sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
weight_study = optuna.create_study(direction="maximize", sampler=weight_sampler)

if WEIGHT_TUNE:

    # Run the optimization process
    weight_study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=300)

    # Get the best parameters after optimization
    weight_best_params = weight_study.best_params

[I 2024-03-08 16:31:29,834] A new study created in memory with name: no-name-2fd742c0-8323-4bc1-9833-0b8fc3b5ae71
/opt/anaconda3/envs/cse/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [16:31:30] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


In [29]:
voting_estimators = [
    ('XGB', XGBClassifier(**xgb_params)),
    ('LGBM', LGBMClassifier(**lgbm_params)),
    ('CAT', CatBoostClassifier(**cat_params))
]

In [33]:
voting_classifier = VotingClassifier(
    estimators=voting_estimators,
    voting='soft',
    weights=[
            3.9963209507789, 4.765892955447102,1.530989398169959
        ],
    )
    
voting_classifier.fit(train_x, train_y)

predicted_probs = voting_classifier.predict_proba(test_x)[:, :-1] # Filtering out 8th's target (No Faults)

/opt/anaconda3/envs/cse/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [13:51:48] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


KeyError: 7

#### Catboost

In [ ]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(task_type = 'GPU',iterations=320,learning_rate= 4.5e-2,depth= 13, l2_leaf_reg=1e-3,bootstrap_type="Bayesian",random_strength=1e-8,bagging_temperature=0.5,od_wait=10,verbose=False)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)
accuracy_score(test_y, y_pred)




In [23]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)


[W 2024-03-08 08:51:31,684] Trial 0 failed with parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26} because of the following error: CatBoostError("/src/catboost/catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'").
Traceback (most recent call last):
  File "/opt/anaconda3/envs/cse/lib/python3.12/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_45788/1733134025.py", line 17, in objective
    model.fit(train_x, train_y)
  File "/opt/anaconda3/envs/cse/lib/python3.12/site-packages/catboost/core.py", line 5201, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseli

CatBoostError: /src/catboost/catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'

In [318]:
y_prob = modelcat.predict(test_x)
accuracy = accuracy_score(test_y, y_prob)
print('Accuracy:', accuracy)

Accuracy: 0.9945486021584589


#### The rest

In [149]:
%%bash
sudo mv /home/baki/Downloads/test.csv.zip /home/baki/Desktop/cse_hackathon/

In [156]:
!unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [15]:
test = pd.read_csv('test.csv')

In [263]:
test['date expédition'] = pd.to_datetime(df_train['date expédition'])
test['date expédition']

0       2024-01-10 18:18:53
1       2024-01-04 18:55:07
2       2024-01-02 16:47:55
3       2024-01-01 17:27:33
4       2024-01-07 14:45:47
                ...        
67866   2024-01-07 19:37:24
67867   2024-01-11 15:47:07
67868   2024-01-12 18:41:00
67869   2024-01-07 11:17:12
67870   2024-01-11 17:38:40
Name: date expédition, Length: 67871, dtype: datetime64[ns]

In [16]:
testing = process(test)

In [17]:
to_predict = testing.drop(['Tracking','date livraison/échec','date dernier statut', 'date expédition', 'date première tentative'], axis=1)

In [44]:
to_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67871 entries, 0 to 67870
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID expediteur                      67871 non-null  int64  
 1   depart commune                     67871 non-null  int64  
 2   depart wilaya                      67871 non-null  int64  
 3   stop desk?                         67871 non-null  int64  
 4   destination commune                67871 non-null  int64  
 5   destination wilaya                 67871 non-null  int64  
 6   yeardate expédition                67871 non-null  float64
 7   monthdate expédition               67871 non-null  int32  
 8   daydate expédition                 67871 non-null  int32  
 9   weekdaydate expédition             67871 non-null  int32  
 10  weekofyeardate expédition          67871 non-null  UInt32 
 11  hourdate expédition                67871 non-null  int

In [45]:
y_predict = model.predict(to_predict)

In [268]:
xx = pd.DataFrame()
xx['Tracking'] = testing['Tracking']
xx['dernier statut'] = y_predict
xx.to_csv('submission_Yalidine5.csv', index=False)
xx.head(10)

,Tracking,dernier statut
0,yal-ABC256315,3
1,yal-ABC291785,3
2,yal-ABC288610,3
3,yal-ABC196442,3
4,yal-ABC131159,3
5,yal-ABC272768,3
6,yal-ABC119741,3
7,yal-ABC195795,3
8,yal-ABC094490,3
9,yal-ABC214527,3


In [48]:
%%bash
sudo cp submission_Yalidine9.csv /home/baki/Desktop/
sudo chown baki submission_Yalidine4.csv

In [201]:
inversed_predict = LE.inverse_transform(y_predict)

In [36]:
lee = LabelEncoder()
y = lee.fit_transform(df_train['dernier statut'])

In [26]:
df_train['dernier statut']

0         Livré
1         Livré
2         Livré
3         Livré
4         Livré
          ...  
271476    Livré
271477    Livré
271478    Livré
271479    Livré
271480    Livré
Name: dernier statut, Length: 271481, dtype: object

In [46]:
invers = lee.inverse_transform(y_predict)

In [38]:
invers

array(['Livré', 'Livré', 'Livré', ..., 'Livré', 'Livré', 'Livré'],
      dtype=object)

In [47]:
xx = pd.DataFrame()
xx['Tracking'] = testing['Tracking']
xx['dernier statut'] = invers
xx.to_csv('submission_Yalidine9.csv', index=False)
xx.head(10)

,Tracking,dernier statut
0,yal-ABC256315,Livré
1,yal-ABC291785,Livré
2,yal-ABC288610,Livré
3,yal-ABC196442,Livré
4,yal-ABC131159,Livré
5,yal-ABC272768,Livré
6,yal-ABC119741,Livré
7,yal-ABC195795,Livré
8,yal-ABC094490,Livré
9,yal-ABC214527,Livré
